In [ ]:
import geemap
import ee
import numpy as np
from matplotlib.pyplot import cm

In [ ]:
ee.Initialize()

In [ ]:
def get_climate_month(collection, month_no, feature='mean_2m_air_temperature'):
    return collection.filter(ee.Filter.calendarRange(start=month_no, field='month')).select([feature]).mean()

In [ ]:
def calc_trange(image):
    band = image.expression('MAX - MIN',{'MAX': image.select('maximum_2m_air_temperature'),
                                         'MIN':image.select('minimum_2m_air_temperature')})
    return image.addBands(band.select([0], ['range_2m_air_temperature']))

In [ ]:
def calc_wind_speed(image):
    band = image.expression('sqrt((U*U) + (V*V))',{'U': image.select('u_component_of_wind_10m'),
                                         'V':image.select('v_component_of_wind_10m')})
    return image.addBands(band.select([0], ['wind_speed']))

In [ ]:
def calc_relative_humidity(image):
    expression = '(c*b*(TD-T))/((c+T)*(c+TD))'
    band = image.expression(expression,{
        'TD': image.select('dewpoint_2m_temperature').add(OFFSET_T),
        'T':image.select('mean_2m_air_temperature').add(OFFSET_T),
        'b': 243.04,
        'c': 17.625})
    out = ee.Image.exp(band)
    return image.addBands(out.select([0], ['relative_humidity']))

## Settings 

In [ ]:
Rd_Bu = ['#67001f','#b2182b','#d6604d','#f4a582','#fddbc7','#f7f7f7','#d1e5f0','#92c5de','#4393c3','#2166ac','#053061'][::-1]
Blues = ['#fff7fb','#ece7f2','#d0d1e6','#a6bddb','#74a9cf','#3690c0','#0570b0','#045a8d','#023858']
Yl_Bu = ['#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#253494','#081d58']
Greys = ['#ffffff','#f0f0f0','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525','#000000']
Greys_r = Greys[::-1]

In [ ]:
OFFSET_T = 273.15

In [ ]:
vis_params_t = {
  'min': -40 + OFFSET_T,
  'max': 40 + OFFSET_T,
  'palette': Rd_Bu}

In [ ]:
vis_params_trange = {
  'min': 0,
  'max': 40,
  'palette': Rd_Bu}

In [ ]:
vis_params_rh = {
  'min': 0.7**0.5,
  'max': 1**0.5,
  'palette': Rd_Bu[::-1]}

In [ ]:
vis_params_p = {
  'min': 0,
  'max': 0.4,
  'palette': Blues}

In [ ]:
vis_params_wind = {
  'min': 0,
  'max': 10**0.5,
  'palette': Greys_r}

## Load data 

In [ ]:
era = ee.ImageCollection("ECMWF/ERA5/MONTHLY")

In [ ]:
#SETTINGS
climate_period = ee.Filter.date('1981-01-01', '2010-12-31')

In [ ]:
era_climate = era.filter(climate_period).map(calc_trange).map(calc_wind_speed).map(calc_relative_humidity)

## Make video animation 

### Climate 

In [ ]:
video_settings = {
  'dimensions': 768,
  'framesPerSecond': 10}

In [ ]:
aoi_global = {'region': ee.Geometry.BBox(-180,-90, 180, 90)}

#### Temperature 

In [ ]:
feature = 'mean_2m_air_temperature'
climate_collection_t = ee.ImageCollection.fromImages([get_climate_month(era_climate, int(i), feature)for i in np.arange(1, 13)])

In [ ]:
video_params = {**video_settings,**vis_params_t, **aoi_global}
geemap.download_ee_video(climate_collection_t, video_params, 'ERA5_monthly_t.gif')

#### Dewpoint 

In [ ]:
feature='dewpoint_2m_temperature'
climate_collection_dp = ee.ImageCollection.fromImages([get_climate_month(era_climate, int(i), feature)for i in np.arange(1, 13)])

In [ ]:
video_params = {**video_settings,**vis_params_t, **aoi_global}
geemap.download_ee_video(climate_collection_dp, video_params, 'ERA5_monthly_dp.gif')

#### Temperature range 

In [ ]:
feature='range_2m_air_temperature'
climate_collection_trange = ee.ImageCollection.fromImages([get_climate_month(era_climate, int(i), feature)for i in np.arange(1, 13)])

In [ ]:
video_params = {**video_settings,**vis_params_trange, **aoi_global}
geemap.download_ee_video(climate_collection_trange, video_params, 'ERA5_monthly_trange.gif')

#### Precipitation 

In [ ]:
feature = 'total_precipitation'
climate_collection_p = ee.ImageCollection.fromImages([get_climate_month(era_climate, int(i), feature)for i in np.arange(1, 13)])

In [ ]:
video_params = {**video_settings,**vis_params_p, **aoi_global}
geemap.download_ee_video(climate_collection_p, video_params, 'ERA5_monthly_p.gif')

#### Wind speed 

In [ ]:
# power 0.5
feature = 'wind_speed'
climate_collection_w = ee.ImageCollection.fromImages([(get_climate_month(era_climate, int(i), feature)).pow(ee.Image.constant(0.5)) for i in np.arange(1, 13)])

In [ ]:
video_params = {**video_settings,**vis_params_wind, **aoi_global}
geemap.download_ee_video(climate_collection_w, video_params, 'ERA5_monthly_windspeed_power.gif')

In [ ]:
aoi_CAm = {'region':ee.Geometry.BBox(-120, -10, -50, 40)}
video_params = {**video_settings,**vis_params_wind, **aoi_CAm}
geemap.download_ee_video(climate_collection_w, video_params, 'ERA5_monthly_windspeed_power_aoi_CAm.gif')

In [ ]:
aoi_CAs = {'region':ee.Geometry.BBox(20, 10, 90, 50)}
video_params = {**video_settings,**vis_params_wind, **aoi_CAs}
geemap.download_ee_video(climate_collection_w, video_params, 'ERA5_monthly_windspeed_power_aoi_CAs.gif')

#### Relative Humidity

In [ ]:
feature = 'relative_humidity'
climate_collection_rh = ee.ImageCollection.fromImages([get_climate_month(era_climate, int(i), feature).pow(ee.Image.constant(0.5)) for i in np.arange(1, 13)])

In [ ]:
video_params = {**video_settings,**vis_params_rh, **aoi_global}
geemap.download_ee_video(climate_collection_rh, video_params, 'ERA5_monthly_rh.gif')